# Forecast Performance Scorecard

This scorecard provides a systematic, top-down approach to analyze forecast performance and identify areas for model improvement. The process follows a drill-down methodology from high-level business metrics to individual product analysis.

## Scorecard Process Flow:

### 1. **Portfolio Overview** 
Start with the big picture - how is the entire business performing against forecasts? This gives you the overall health of your forecasting system and identifies if there are systematic issues affecting the whole portfolio.

### 2. **Category Performance Analysis**
Drill down into sales categories (Very Low, Low, Alive, Medium, Winning, High Winning) to identify which product segments are driving forecast errors. This helps prioritize which categories need immediate attention.

### 3. **Error Distribution Deep Dive**
For problematic categories, analyze the distribution of forecast errors to understand:
- Are we consistently over-forecasting or under-forecasting?
- What's the magnitude of errors?
- Which products are the biggest contributors to poor performance?

### 4. **Individual Product Investigation**
Focus on the worst-performing products to understand root causes:
- Are there data quality issues?
- Is the product experiencing unusual patterns (promotions, stockouts, seasonality)?
- Are there external factors affecting demand?

### 5. **Actionable Insights Generation**
Based on the analysis, identify specific actions:
- Which products need model retraining?
- What data quality issues need fixing?
- Which categories need different forecasting approaches?
- What external factors should be incorporated into the model?

## How to Use This Scorecard:

1. **Run the complete notebook** to get the full analysis
2. **Start with the business-level view** to understand overall performance
3. **Identify problematic categories** using the error distribution plots
4. **Drill down into specific products** that are driving poor performance
5. **Document findings** walk through lots of skus and identofy dominnat issues then correct. Don't focus on correcting specific edge cases. This will end up messing working cases



# Scorecard

In [1]:
# =============================================================================
# ALL IMPORTS - CONSOLIDATFEVAL IN FIRST CELL
# =============================================================================

# Standard library imports
import sys
import importlib
from importlib import reload

# Third-party imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

# Local imports
import configuration as CONFIG
from functions_library.supabase_connection import SupabaseClient
import functions_library.forecast_evaluation as FEVAL

# =============================================================================
# MODULE RELOADING - Ensure latest versions are loaded
# =============================================================================
importlib.reload(CONFIG)
importlib.reload(sys.modules['functions_library.supabase_connection'])
importlib.reload(sys.modules['functions_library.forecast_evaluation'])

# =============================================================================
# SPARK INITIALIZATION
# =============================================================================
spark = SparkSession.builder.appName('ForecastEvaluation').master('local[1]').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# =============================================================================
# SUPABASE CLIENT INITIALIZATION
# =============================================================================
storage_client = SupabaseClient()

In [119]:
EVALUATION_START_DATE = "2025-10-07"
EVALUATION_END_DATE = "2025-10-17"

In [120]:
products = storage_client.batch_read_table("products", spark)
prod_day_history = storage_client.download_processed_features(spark)
prod_day_forecast = FEVAL.load_latest_per_date_forecasts(storage_client, spark)


In [121]:
evaluation_data = (prod_day_forecast
                   .join(prod_day_history.select("product_id", "date", "sales_units"), on=["product_id", "date"], how="inner")
                   .groupBy("product_id", "style", "age_sales_category", "age_category", "sales_category", "forecast_method")
                   .agg(F.sum("sales_units").alias("sales_units"),
                        F.sum("forecast").alias("forecast")))
evaluation_data = FEVAL.calculate_bias_and_error_bins(evaluation_data)

In [122]:
validation_data = (prod_day_history
                   .join(prod_day_forecast.select("product_id", "date", "forecast_method", "forecast"), on=["product_id", "date"],how="full"))

# Forward fill style column by product_id
window = Window.partitionBy("product_id").orderBy("date")
validation_data = validation_data.withColumn("style", F.last("style", ignorenulls=True).over(window))

validation_data = FEVAL.calculate_bias_and_error_bins(validation_data)

# Business Summary

In [123]:
pdf, plot = FEVAL.plot_business_level(validation_data, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()

Evaluation frame (2025-10-07 - 2025-10-17):
  Total Sales: 4817.0
  Total Forecast: 8335.916494734924
  Over/Under Forecast: 3518.9164947349236 units (73.1%)


# Overall Evaluation

In [124]:
# Define per-bin good thresholds (understock / overstock)
GOOD_THRESHOLD = {
    "02| Very Low": (5, 10),
    "03| Low": (5, 10),
    "04| Alive": (5, 20),
    "05| Medium": (5, 20),
    "06| Winning": (5, 20), # Bias
    "07| High Winning": (5, 30) # Bias
}

In [125]:
df, plot = FEVAL.plot_attribute_distribution(evaluation_data, ["sales_category"])
plot.show()



In [126]:
FEVAL.plot_accuracy_distribution(evaluation_data, method="bias").show()
FEVAL.plot_accuracy_distribution(evaluation_data, method="error").show()

In [127]:
df = (evaluation_data
    .where(~(F.col("sales_category").isin(["02| Very Low", "03| Low", "07| High Winning"]))))
FEVAL.plot_accuracy_distribution(df, method="bias").show()

# Investigate Sales Categories

In [128]:
SELECTFEVAL_SALES_BIN = "07| High Winning" 
METHOD = "bias"

In [129]:
evaluation_data_category = evaluation_data.where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
FEVAL.analyze_category_performance(evaluation_data_category)

,product_count,sales_units,forecast,error
0,9,704,1345.724465,641.724465


In [130]:
FEVAL.plot_accuracy_distribution(evaluation_data_category, method=METHOD).show()

# Investigate Products


In [131]:
SELECTFEVAL_BIAS_BIN = "22| > +100"
LIMIT = 10

In [132]:
(evaluation_data_category
    .where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
    # .where(F.col("bias_bin").rlike(SELECTFEVAL_BIAS_BIN))
    .orderBy(F.desc("error")).limit(LIMIT).toPandas())

,product_id,style,age_sales_category,age_category,sales_category,forecast_method,sales_units,forecast,bias,error,bias_bin,error_bin
0,9c98308b-05cc-44e4-b40b-116d20efd49f,MW1604,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,72,218.454856,2.034095,146.454856,22| > +100,22| > +50
1,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,MW1537,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,147,253.121437,0.721915,106.121437,19| +70 +80,22| > +50
2,6496ce7e-1a2b-421e-9d64-ef17ec68903c,MWT405,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,83,165.939084,0.999266,82.939084,21| +90 +100,22| > +50
3,1392c7f9-172f-4069-9b61-f9504c7ee281,MW1545,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,58,137.211508,1.365716,79.211508,22| > +100,22| > +50
4,1ac0afb5-8fa6-4507-857b-c6d95655c103,MW1384,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,101,177.552500,0.757946,76.552500,19| +70 +80,22| > +50
5,0e0a1c0d-6a98-4d33-b2dc-66d810894266,MW1341,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,82,158.045173,0.927380,76.045173,21| +90 +100,22| > +50
6,5efc1f29-afcf-40b2-96b4-35d25dd411e0,MWD04,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,108,137.490459,0.273060,29.490459,14| +20 +30,17| +25 +30
7,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,MW1537,04| Mature_03| High,04| Mature,07| High Winning,prophet,6,22.248294,2.708049,16.248294,22| > +100,15| +15 +20
8,6730ba30-4cdb-4afe-81f9-5e962c3fb464,MWD09,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,20,32.612101,0.630605,12.612101,18| +60 +70,14| +10 +15
9,5efc1f29-afcf-40b2-96b4-35d25dd411e0,MWD04,04| Mature_03| High,04| Mature,07| High Winning,prophet,7,13.245906,0.892272,6.245906,20| +80 +90,13| +5 +10


In [133]:
# Example usage
SELECTED_STYLE = "MW1604" 
pdf_product, plot = FEVAL.plot_product_details_daily_short(validation_data, SELECTED_STYLE, products_df=products, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()


Product Title: MW1604 -  عباية سوداء بتطريز ورود على الجوانب | Black Abaya with Floral Side Embroidery
Price: 252.17
Status: ACTIVE


In [135]:
prod_day_forecast.where(F.col("style") == SELECTED_STYLE).orderBy("date").toPandas()

,date,product_id,age_sales_category,age_category,sales_category,style,forecast_method,forecast,run_id
0,2025-10-07,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,24.872570,1
1,2025-10-08,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,21.259828,1
2,2025-10-09,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,20.265093,1
3,2025-10-10,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,21.336500,1
4,2025-10-11,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,21.384643,1
5,2025-10-12,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,22.298784,1
6,2025-10-13,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,21.810831,1
7,2025-10-14,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,23.024241,2
8,2025-10-15,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,22.441472,2
9,2025-10-16,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1604,prophet,19.760894,2


In [ ]:
# prod_day_history.where(F.col("style") == SELECTED_STYLE).orderBy("date").toPandas()

In [ ]:
prod_day_forecast

DataFrame[date: date, product_id: string, age_sales_category: string, age_category: string, sales_category: string, style: string, forecast_method: string, forecast: double, run_id: int]

In [ ]:
# Load forecast for run_id 10 from storage
print("Loading forecast data for run_id 10 from storage...")

# Load the forecast data for run_id 10
run_id_10_forecast = storage_client.download_forecast_run(run_id=10, spark=spark)
run_id_10_forecast.where(F.col("style") == SELECTED_STYLE).orderBy("date").toPandas()


Loading forecast data for run_id 10 from storage...


,product_id,age_sales_category,age_category,sales_category,style,forecast_method,date,forecast
0,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-17,20.025773
1,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-18,22.014546
2,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-19,22.381734
3,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-20,19.586539
4,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-21,20.575907
5,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-22,20.684719
6,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-23,18.302722
7,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-24,21.734285
8,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-25,23.870116
9,9c98308b-05cc-44e4-b40b-116d20efd49f,04| Mature_03| HIGH,04| Mature,07| High Winning,MW1604,prophet,2025-10-26,24.245808
